# Package Config

In [1]:
from plotly.offline import init_notebook_mode, iplot # from plotly.offline import init_notebook_mode, iplot # 
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)

import numpy as np
from scipy.integrate import odeint

# Load Models

In [2]:
%run AgentDynamics.ipynb

# Simulation

In [9]:
# System settings

n = 2
N = 3
nStates = n*2

AgentPars = {
    'n'  : n,
    'N'  : N,
    'k_i': 1,
    'm_i': 1,
}

agents = list(range(0,N))

# Axes
vertical   = 0
horizontal = 1

# Time settings
dt    = 0.5*5                        # Networked Controller sampling time
odesteps_per_timesample = 10       # Number of ODE integration steps per discrete time sample
t_end = 20                         # Simulation time
t_discrete = np.arange(0,t_end,dt) # Discrete time sample vector

In [10]:
def get_s_dot(s,t,u):
    s_i = [s[i*nStates:(i+1)*nStates]  for i in agents]
    s_dot_i = [get_s_dot_i(t, s_i[i], u_i[i], AgentPars)   for i in agents]
    return np.concatenate(s_dot_i)

# Initial Conditions
s_start = 2.5 * np.random.randn(2*n*N) + 3

# Initialize empty result vectors
t_history = np.empty(shape=[1,      0 ]) 
s_history = np.empty(shape=[N*nStates, 0]) 
end = -1

# At every sample do
for tnow in t_discrete:
    
    # List of all the control inputs
    
     
    r_i = [s_start[2*i*n:(2*i+1)*n] for i in agents]
    
    u_i = [-1*r_i[i] for i in agents]
        
    t_sim = np.linspace(tnow,tnow+dt,odesteps_per_timesample)
    s_sim = odeint(get_s_dot, s_start, t_sim, args=(u_i,)).T
    
    ## The final y state is the initial state for next time
    s_start = s_sim[:,end]
       
    # Append the results
    t_history = np.append(t_history,t_sim)
    s_history = np.append(s_history,s_sim, axis = horizontal)

    
## Match time and output signals for plotting    
posx_plot = Scatter(x = t_history, y = s_history[0, :], name = 'position x_1')   
posy_plot = Scatter(x = t_history, y = s_history[1, :], name = 'position y_2')   



position_traces = [Scatter(x = s_history[i*nStates, :], y = s_history[i*nStates+1, :], name = 'Agent ' + str(i))  for i in agents]
    
# Plot results inline    
iplot([posx_plot,posy_plot])
iplot(position_traces)


print(t_sim)

[ 17.5         17.77777778  18.05555556  18.33333333  18.61111111
  18.88888889  19.16666667  19.44444444  19.72222222  20.        ]


In [8]:
s_sim

array([[ 1.62888616,  1.54998655,  1.38048348,  1.14235148,  0.85223542,
         0.52274327,  0.1634251 , -0.21848525, -0.61750838, -1.02949385],
       [ 2.62010415,  2.61947879,  2.4424866 ,  2.13190271,  1.72012772,
         1.23170403,  0.68522161,  0.09476172, -0.52900956, -1.17801307],
       [-0.08861785, -0.46218662, -0.74515193, -0.9594883 , -1.12184063,
        -1.24481685, -1.33796706, -1.40852509, -1.46197033, -1.50245324],
       [ 0.37815075, -0.3490306 , -0.89984511, -1.31706793, -1.63309965,
        -1.87248267, -2.05380696, -2.19115378, -2.29518921, -2.37399241],
       [ 1.71955755,  2.10366531,  2.27876558,  2.29555001,  2.19241564,
         1.998447  ,  1.73567456,  1.42078563,  1.06642028,  0.68215291],
       [ 1.20613756,  1.50216419,  1.64513559,  1.67217302,  1.61139447,
         1.48409839,  1.30641761,  1.09057216,  0.8458183 ,  0.57916733],
       [ 1.83359195,  0.97182932,  0.31907417, -0.17536513, -0.54988564,
        -0.83357187, -1.04845431, -1.21122025